The deadline for this homework is on **21.03.2025 18:29** (right before the practice session). After completing the exercises, you should

1. Download this file into your computer (`File` $\to$ `Download .ipynb`)

2. Name the file in the following way *HWx_NameSurname* (for example `HW3_NshanPotikyan.ipynb`)

4. Submit the file via the e-learning environment.

**Note** if you do not follow any of the above conditions, your homework will not be graded.

**Problem.** During the practice session we tried to build a binary classifier on the adult dataset which is highly imbalanced.

* In this homework, you need to take the same dataset but this time you need to

 * use more features from the original data
 * try different sampling techniques from [imblearn](https://imbalanced-learn.org/stable/references/index.html) to tackle the class imbalance problem
 * experiment with different ensemble methods (the ones that we have discussed so far) to beat the score we got during the practice session

* Evaluate the model performance in terms of the accuracy score.

* Use the best data processing method to train a final model and report the accuracy score on the test dataset.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.ensemble import StackingClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline


In [1]:
!curl https://archive.ics.uci.edu/static/public/2/adult.zip -o adult.zip
!unzip adult.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  605k    0  605k    0     0   363k      0 --:--:--  0:00:01 --:--:--  363k
Archive:  adult.zip
  inflating: Index                   
  inflating: adult.data              
  inflating: adult.names             
  inflating: adult.test              
  inflating: old.adult.names         


In [2]:
data = pd.read_csv('adult.data', header=None, na_values=["NA", " ?", ""])
data.columns = ["age", "workclass", "not_needed1", "education1",
                "education", "marital_status","occupation",
                "relationship", "race", "sex", "capital_gain",
                "capital_loss", "hours_per_week", "country", "income"]
data.dropna(inplace=True)
data.head()

,age,workclass,not_needed1,education1,education,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [3]:
data.isna().sum()

age               0
workclass         0
not_needed1       0
education1        0
education         0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
country           0
income            0
dtype: int64

In [4]:
def relationship_group(rel):
    if rel in [' Husband', ' Wife']:
        return 'Spouse'
    elif rel == ' Own-child':
        return 'Child'
    elif rel in [' Not-in-family', ' Unmarried', ' Other-relative']:
        return 'Other'
    else:
        return None

def country_group(x):
    x = x.strip()  # remove leading/trailing spaces
    if x == 'United-States':
        return 'United-States'
    elif x in ['Canada', 'Outlying-US(Guam-USVI-etc)']:
        return 'North America'
    elif x in ['Cuba', 'Jamaica', 'Mexico', 'Puerto-Rico', 'Honduras', 
               'Dominican-Republic', 'El-Salvador', 'Guatemala', 'Nicaragua',
               'Trinadad&Tobago', 'Haiti', 'Ecuador', 'Peru', 'Columbia']:
        return 'Latin America/Caribbean'
    elif x in ['India', 'Iran', 'Philippines', 'Cambodia', 'Thailand', 
               'Laos', 'Taiwan', 'China', 'South', 'Japan', 'Vietnam', 'Hong']:
        return 'Asia'
    elif x in ['England', 'Germany', 'France', 'Italy', 'Scotland', 'Poland',
               'Portugal', 'Yugoslavia', 'Greece', 'Ireland', 'Hungary',
               'Holand-Netherlands']:
        return 'Europe'
    else:
        return 'Other'

data['native_country_group'] = data['country'].apply(country_group)
data['relationship_group'] = data['relationship'].apply(relationship_group)

In [5]:
data.drop(["relationship", 'not_needed1', 'education1', 'country'], axis = 1, inplace = True)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(["income"], axis = 1),
                                                    data["income"],
                                                    test_size = 0.2,
                                                    random_state = 0)

In [7]:
print(y_train.value_counts())
print(y_test.value_counts())

income
<=50K    18122
>50K      6007
Name: count, dtype: int64
income
<=50K    4532
>50K     1501
Name: count, dtype: int64


In [8]:
X_train

,age,workclass,education,marital_status,occupation,race,sex,capital_gain,capital_loss,hours_per_week,native_country_group,relationship_group
30461,34,Private,10,Never-married,Adm-clerical,White,Male,0,0,40,United-States,Child
18186,51,Private,9,Divorced,Craft-repair,White,Male,0,0,40,United-States,Other
24974,50,Private,9,Widowed,Prof-specialty,Black,Female,0,0,40,United-States,Other
25659,52,Self-emp-not-inc,10,Widowed,Craft-repair,Black,Male,0,0,35,United-States,Other
13876,31,Private,9,Married-civ-spouse,Machine-op-inspct,White,Male,0,0,40,United-States,Spouse
...,...,...,...,...,...,...,...,...,...,...,...,...
14169,66,Local-gov,16,Divorced,Prof-specialty,White,Female,3273,0,40,United-States,Other
21206,50,Private,14,Married-civ-spouse,Exec-managerial,White,Male,0,0,50,United-States,Spouse
10646,50,Private,13,Divorced,Sales,White,Male,0,0,45,United-States,Other
11687,49,Self-emp-not-inc,13,Married-civ-spouse,Exec-managerial,White,Male,0,0,60,United-States,Spouse


In [9]:
numerical_features = ["age", "capital_gain", "capital_loss", "hours_per_week"]
categorical_features = ["workclass", "marital_status", "occupation", "race", "sex", "relationship_group", "native_country_group"]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([('scaler', StandardScaler()),('poly', PolynomialFeatures(degree=2))]), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [10]:
bagging_model = BaggingClassifier(estimator=DecisionTreeClassifier(),
                                  n_estimators=100,
                                  random_state=42)

random_forest_model = RandomForestClassifier(n_estimators=100,
                                             random_state=42)

weighted_voting_model = VotingClassifier(
    estimators=[('bagging', bagging_model),
                ('random_forest', random_forest_model)],
    voting='soft')

base_models = [('bagging', bagging_model),
               ('random_forest', random_forest_model)]
stacking_model = StackingClassifier(estimators=base_models,
                                    final_estimator=DecisionTreeClassifier())


In [11]:
bagging_model.fit(X_train, y_train)
random_forest_model.fit(X_train, y_train)
weighted_voting_model.fit(X_train, y_train)
stacking_model.fit(X_train, y_train)


bagging_pred = bagging_model.predict(X_test)
random_forest_pred = random_forest_model.predict(X_test)
weighted_voting_pred = weighted_voting_model.predict(X_test)
stacking_pred = stacking_model.predict(X_test)

bagging_accuracy = accuracy_score(y_test, bagging_pred)
random_forest_accuracy = accuracy_score(y_test, random_forest_pred)
weighted_voting_accuracy = accuracy_score(y_test, weighted_voting_pred)
stacking_accuracy = accuracy_score(y_test, stacking_pred)

print("Bagging Accuracy:", bagging_accuracy)
print("Random Forest Accuracy:", random_forest_accuracy)
print("Weighted Voting Accuracy:", weighted_voting_accuracy)
print("Stacking Accuracy:", stacking_accuracy)

Bagging Accuracy: 0.8340792308967346
Random Forest Accuracy: 0.8403779214321233
Weighted Voting Accuracy: 0.8398806563898558
Stacking Accuracy: 0.8012597381070777


## Handling class Imbalance

In [12]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

over = SMOTE(sampling_strategy=0.5)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train, y_train)


In [13]:
# Train the models
bagging_model.fit(X_train_resampled, y_train_resampled)
random_forest_model.fit(X_train_resampled, y_train_resampled)
weighted_voting_model.fit(X_train_resampled, y_train_resampled)
stacking_model.fit(X_train_resampled, y_train_resampled)

# Evaluate the models
bagging_pred = bagging_model.predict(X_test)
random_forest_pred = random_forest_model.predict(X_test)
weighted_voting_pred = weighted_voting_model.predict(X_test)
stacking_pred = stacking_model.predict(X_test)

# Calculate and print the accuracy for each model
bagging_accuracy = accuracy_score(y_test, bagging_pred)
random_forest_accuracy = accuracy_score(y_test, random_forest_pred)
weighted_voting_accuracy = accuracy_score(y_test, weighted_voting_pred)
stacking_accuracy = accuracy_score(y_test, stacking_pred)

print("Bagging Accuracy:", bagging_accuracy)
print("Random Forest Accuracy:", random_forest_accuracy)
print("Weighted Voting Accuracy:", weighted_voting_accuracy)
print("Stacking Accuracy:", stacking_accuracy)

Bagging Accuracy: 0.825957235206365
Random Forest Accuracy: 0.8385546162771423
Weighted Voting Accuracy: 0.8359025360517156
Stacking Accuracy: 0.8027515332338804
